# Methods
Narrow-band extraction of Lyman alpha emission at 4 

In [3]:
import kcwitools.io as kio
import kcwitools.utils as ku
from astropy.io import fits, ascii
import numpy as np
import matplotlib.pyplot as plt
from kcwitools.image import build_whitelight
from matplotlib.widgets import Slider, Button, TextBox, RangeSlider

f=fits.open("/Users/robertseaton/Desktop/Physics-NCState/---Research/FITS data/J1429/SGAS1429+1202.fits")

hdr=f[0].header
data=f[0].data
wave=ku.build_wave(hdr)
wl_image=build_whitelight(hdr, data)

In [4]:
# %matplotlib

In [5]:
f.info()

Filename: /Users/robertseaton/Desktop/Physics-NCState/---Research/FITS data/J1429/SGAS1429+1202.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      22   (55, 64, 2000)   float64   


In [6]:
wave

array([3502.00167624, 3503.00193315, 3504.00219007, ..., 5499.52757384,
       5500.52784093, 5501.52810801])

In [7]:
wave.size

2000

In [8]:
# This just draws a box centered at (x,y) and sz from that center point in the n/e/s/w directions 
def plotbox(plt, x, y, labels, align, sz, c):
    for i in range(len(x)):
        plt.plot(
            [x[i]-sz, x[i]-sz, x[i]+sz, x[i]+sz, x[i]-sz], 
            [y[i]-sz, y[i]+sz, y[i]+sz, y[i]-sz, y[i]-sz], 
            '-', color=c)
        ha_ = align[i][0]
        va_ = align[i][1]
        plt.text(x[i], y[i]+1.5*sz, labels[i], color=c);


Show SGAS1429+1202 with callout boxes

In [12]:
# Show the SGAS1429 image

# %matplotlib
x_cen = [20, 25, 39, 24];
y_cen = [35, 36, 29, 21];
labels = ["1","2","3","4"];
alignment = [ # [ha, va] :: ha={'left', 'center', 'right'} va={'bottom', 'baseline', 'center', 'center_baseline', 'top'}
    ['center', 'top'],
    ['center', 'top'],
    ['center', 'top'],
    ['center', 'top']
]; 

# fig = plt.figure(figsize=(6,6))
# fig, ax = plt.subplots(3, 3)
fig = plt.figure()
grid_size = (4,6)
fig = plt.figure(figsize=(14,8))

ax0 = plt.subplot2grid(grid_size, (1, 0), colspan=2, rowspan=2)
ax = [];
ax.append(plt.subplot2grid(grid_size, (0, 2), colspan=4))
ax.append(plt.subplot2grid(grid_size, (1, 2), colspan=4))
ax.append(plt.subplot2grid(grid_size, (2, 2), colspan=4))
ax.append(plt.subplot2grid(grid_size, (3, 2), colspan=4))
# ax.append(plt.subplot2grid(grid_size, (4, 2), colspan=4))

# source wavelength range
slower_val = 4599; # source lower bound (in angstroms)
supper_val = 4702; # source lower bound (in angstroms)
source_diff = supper_val - slower_val;

# continuum wavelength range
# Ahmed says that the continuum range needs to be roughly equal to the source range
clower_val = 4115; # continuum lower bound (in angstroms)
cupper_val = clower_val + source_diff; # continuum upper bound (in angstroms)

# Create the RangeSlider
sslider_ax = fig.add_axes([0.20, 0.1, 0.60, 0.03])
sslider = RangeSlider(sslider_ax, "Source Range::", wave.min(), wave.max(), valinit=(slower_val, supper_val))

# Create the Continuum Slider
cslider_ax = fig.add_axes([0.20, 0.1, 0.60, 0.03])
cslider = RangeSlider(sslider_ax, "Continuum::", wave.min(), wave.max(), valinit=(slower_val, supper_val))

scolor = '#549c54';
ccolor = '#9c5954'; 

slines_l = [];
slines_u = [];
sbounds=[];
slines_l.append(ax[0].axvline(sslider.val[0], color='k'));
slines_u.append(ax[0].axvline(sslider.val[1], color='k'));
slines_l.append(ax[1].axvline(sslider.val[0], color='k'));
slines_u.append(ax[1].axvline(sslider.val[1], color='k'));
slines_l.append(ax[2].axvline(sslider.val[0], color='k'));
slines_u.append(ax[2].axvline(sslider.val[1], color='k'));
slines_l.append(ax[3].axvline(sslider.val[0], color='k'));
slines_u.append(ax[3].axvline(sslider.val[1], color='k'));
sbounds.append(ax[0].axvspan(sslider.val[0], sslider.val[1], facecolor=scolor)); # lower source bounds
sbounds.append(ax[1].axvspan(sslider.val[0], sslider.val[1], facecolor=scolor)); # lower source bounds
sbounds.append(ax[2].axvspan(sslider.val[0], sslider.val[1], facecolor=scolor)); # lower source bounds
sbounds.append(ax[3].axvspan(sslider.val[0], sslider.val[1], facecolor=scolor)); # lower source bounds
clines_l = [];
clines_u = [];
cbounds=[];

# ax[j].axvspan(sslider.val[0], sslider.val[1], facecolor='#2ca02c')

ax[0].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)
ax[1].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)
ax[2].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)

# Create the Vertical lines on the histogram
def set_xvalues(polygon, x0, x1):
    _ndarray = polygon.get_xy()
    _ndarray[:, 0] = [x0, x0, x1, x1, x0]
    polygon.set_xy(_ndarray)
    
# print("lower_limit: ", upper_limit_line, " low: ", lower)
def update_sourceRange(val):
    print("inside update_sourceRange() ");
    print("   val[0]: ", val[0]);
    print("   val[1]: ", val[1]);        
    # The val passed to a callback by the RangeSlider will
    # be a tuple of (min, max)

    for j in range(4):
        set_xvalues(sbounds[j], val[0],val[1]);
        slines_l[j].set_xdata([val[0], val[0]])
        slines_u[j].set_xdata([val[1], val[1]])

    # Redraw the figure to ensure it updates
    fig.canvas.draw_idle()

sslider.on_changed(update_sourceRange)


ax0.imshow(wl_image,origin="lower",interpolation="nearest",cmap="gnuplot",vmin=0)
plotbox(ax0, x_cen, y_cen, labels, alignment, 2, 'c')

colors = ['r','g','b','y'];
j = 0
for j in range(4):
    flux = []
    for i in range(wave.size):
        # print("i: ",i)
        # print("wavelength: ",wave[i], "flux: ",data[i][y][x])
        flux.append(data[i][y_cen[j]][x_cen[j]])
    ax[j].plot(wave, flux, '-', color='black')


plt.subplots_adjust(left=0, bottom=0.16, right=0.979, top=0.94, wspace=0.2, hspace=0.021)
plt.show()
    

In [11]:
# ku.airtovac(wave)